In [10]:
import pandas as pd
import re
import os # 파일 존재 여부 확인용

# --- 1. 엑셀 로드 및 통합 함수 ---
def load_and_combine_excel_sheets(file_path):
    """
    하나의 엑셀(.xlsx) 파일을 직접 로드하여 모든 시트를
    하나의 DataFrame으로 합칩니다. (예: '05년', '06년'...)
    """
    print(f"엑셀 파일 로드 중: {file_path}")
    
    # 엑셀 파일 존재 확인
    if not os.path.exists(file_path):
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        print("스크립트와 동일한 폴더에 엑셀 파일이 있는지 확인하세요.")
        return pd.DataFrame()

    try:
        # sheet_name=None : 엑셀의 모든 시트를 딕셔너리로 불러옵니다.
        # header=2 : 실제 데이터가 3번째 행(인덱스 2)부터 시작합니다.
        all_sheets_dict = pd.read_excel(file_path, sheet_name=None, header=2, engine='openpyxl')
    except Exception as e:
        print(f"엑셀 파일 로드 중 오류 발생: {e}")
        return pd.DataFrame()

    all_dfs = []
    
    # 각 시트('05년', '06년'...)를 순회하며 '연도' 컬럼 추가 후 리스트에 저장
    for sheet_name, df in all_sheets_dict.items():
        match = re.search(r'(\d{2})년', sheet_name)
        if match:
            year = 2000 + int(match.group(1))
            df['연도'] = year
            all_dfs.append(df)
        else:
            print(f"'{sheet_name}' 시트는 'XX년' 패턴이 아니라서 건너뜁니다.")

    if not all_dfs:
        print("처리할 데이터 시트가 없습니다.")
        return pd.DataFrame()
        
    # 모든 시트의 DataFrame을 하나로 합칩니다.
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print("엑셀의 모든 시트 통합 완료.\n")
    return combined_df

# --- 2. 데이터 정리 함수 (Tidy Data) ---
def clean_and_tidy_data(combined_df):
    """
    합쳐진 데이터를 '긴 형식(Tidy Format)'으로 정리합니다.
    (그래프 및 피벗 테이블 생성에 용이한 형태)
    """
    print("데이터 정리 및 '긴 형식'으로 변환 중...")
    
    # 1. 요약 행 ('(년누계)', '년월계') 및 불필요한 행 제거
    undesired_rows = ['(년누계)', '년월계']
    cleaned_df = combined_df[~combined_df['구분'].isin(undesired_rows)]
    cleaned_df = cleaned_df.dropna(subset=['구분']) # '구분'이 비어있는 행 제거

    # 2. 컬럼 정리
    cleaned_df = cleaned_df.rename(columns={'구분': '지역'})
    cleaned_df = cleaned_df.drop(columns=['계'], errors='ignore') # '계' (연간합계) 컬럼 제거
    cleaned_df = cleaned_df.dropna(axis=1, how='all') # 빈 컬럼 제거

    # 3. '넓은 형식' -> '긴 형식'으로 변환 (Melt)
    monthly_columns = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']
    valid_monthly_cols = [col for col in monthly_columns if col in cleaned_df.columns]

    tidy_df = cleaned_df.melt(
        id_vars=['연도', '지역'],
        value_vars=valid_monthly_cols,
        var_name='월',
        value_name='착공실적'
    )
    
    # 4. 데이터 타입 정리
    tidy_df['월'] = tidy_df['월'].str.replace('월', '').astype(int)
    tidy_df['착공실적'] = pd.to_numeric(tidy_df['착공실적'], errors='coerce')
    tidy_df = tidy_df.dropna(subset=['착공실적']) # 착공실적이 없는 (NaN) 데이터 제거
    tidy_df['착공실적'] = tidy_df['착공실적'].astype(int)

    return tidy_df.sort_values(by=['연도', '지역', '월'])

# --- 3. 지역별/전국별 요약표 생성 함수 ---
def create_pivot_tables(tidy_df):
    """
    정리된 '긴 형식' 데이터로 '지역별' 및 '전국' 요약표를 생성합니다.
    """
    print("지역별 및 전국 요약표(피벗 테이블) 생성 중...")
    
    # --- 3-1. 지역별 요약표 ---
    regional_pivot = tidy_df.pivot_table(
        index=['연도', '지역'], # 다중 인덱스: 연도, 그 다음 지역
        columns='월',
        values='착공실적',
        fill_value=0 # 데이터가 없는 칸은 0으로 채움
    )
    # 월 순서 및 요약 컬럼 추가
    monthly_cols = [col for col in range(1, 13) if col in regional_pivot.columns]
    regional_pivot = regional_pivot[monthly_cols]
    regional_pivot['연간 총계'] = regional_pivot[monthly_cols].sum(axis=1)
    regional_pivot['월평균'] = regional_pivot[monthly_cols].mean(axis=1).round(0)
    # 연도와 지역으로 정렬
    regional_pivot = regional_pivot.sort_index(level=['연도', '지역'])

    # --- 3-2. 전국 요약표 (tidy_df에서 다시 계산) ---
    national_monthly_df = tidy_df.groupby(['연도', '월'])['착공실적'].sum().reset_index()
    national_pivot = national_monthly_df.pivot(index='연도', columns='월', values='착공실적').fillna(0)
    # 월 순서 및 요약 컬럼 추가
    monthly_cols = [col for col in range(1, 13) if col in national_pivot.columns]
    national_pivot = national_pivot[monthly_cols]
    national_pivot['연간 총계'] = national_pivot[monthly_cols].sum(axis=1)
    national_pivot['월평균'] = national_pivot[monthly_cols].mean(axis=1).round(0)
    
    return regional_pivot, national_pivot

# --- 메인 실행 ---
if __name__ == "__main__":
    
    # 1. 엑셀 파일 경로 (이 스크립트와 같은 폴더에 있어야 함)
    file_path = r"C:\RAG_COMMANDER\src\data\(05년-24년)착공_연도별,월별,지역별.xlsx"
    
    # 2. 엑셀 로드 및 통합
    combined_data = load_and_combine_excel_sheets(file_path)
    
    if not combined_data.empty:
        # 3. 데이터 정리
        tidy_data = clean_and_tidy_data(combined_data)
        
        # 4. 요약표 생성
        regional_summary, national_summary = create_pivot_tables(tidy_data)
        
        # 5. 모든 지역 데이터를 CSV 파일로 저장 (★중요★)
        regional_csv_path = "regional_housing_starts_summary_05-24.csv"
        national_csv_path = "national_housing_starts_summary_05-24.csv"
        
        # encoding='utf-8-sig' : 엑셀에서 한글이 깨지지 않도록 함
        regional_summary.to_csv(regional_csv_path, encoding='utf-8-sig')
        national_summary.to_csv(national_csv_path, encoding='utf-8-sig')
        
        print(f"\n✅ '전국' 요약표가 {national_csv_path} 로 저장되었습니다.")
        print(f"✅ '모든 지역별' 요약표가 {regional_csv_path} 로 저장되었습니다.")
        print("   (엑셀에서 열어보시면 모든 지역을 확인하실 수 있습니다.)")

        # 6. 콘솔에는 '전국' 요약표와 '지역별' 요약표의 최근 샘플(2023년~)을 출력
        print("\n" + "="*80)
        print("    [전국] 월간 착공실적 요약표 (2015년 이후)")
        print("="*80)
        try:
            # .to_string() : 표를 컬럼에 맞춰 정돈된 문자열로 출력
            print(national_summary.loc[2015:].to_string())
        except KeyError:
            print("'전국' (2015년 이후) 데이터를 찾을 수 없습니다.")

        print("\n" + "="*80)
        print("    [지역별] 월간 착공실적 요약표 (샘플: 2023년~2024년)")
        print("="*80)
        try:
            # 2023년 이후 데이터만 필터링하여 샘플로 출력
            print(regional_summary.loc[2023:].to_string())
        except KeyError:
            print("최근 지역별 데이터를 찾을 수 없습니다.")
    
    else:
        print("엑셀 파일 로드에 실패하여 처리를 중단합니다.")

엑셀 파일 로드 중: C:\RAG_COMMANDER\src\data\(05년-24년)착공_연도별,월별,지역별.xlsx
엑셀의 모든 시트 통합 완료.

데이터 정리 및 '긴 형식'으로 변환 중...
지역별 및 전국 요약표(피벗 테이블) 생성 중...

✅ '전국' 요약표가 national_housing_starts_summary_05-24.csv 로 저장되었습니다.
✅ '모든 지역별' 요약표가 regional_housing_starts_summary_05-24.csv 로 저장되었습니다.
   (엑셀에서 열어보시면 모든 지역을 확인하실 수 있습니다.)

    [전국] 월간 착공실적 요약표 (2015년 이후)
월         1      2      3      4      5      6      7      8      9      10     11      12   연간 총계      월평균
연도                                                                                                         
2015  30502  26920  59057  62956  56135  68134  59122  71032  57354  104356  77552   77050  750170  62514.0
2016  32220  35274  55515  68422  57682  59126  54821  50502  57048   57132  70371   96533  694646  57887.0
2017  28325  42393  43248  42074  37702  44744  40775  53914  48207   35385  46946  104153  567866  47322.0
2018  27078  23427  53420  38873  46330  39055  36462  29555  36301   32248  44885   75770  483404  40284.0
2019  294

In [4]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import json # JSON 파싱 오류 처리를 위해 추가

# --- 1. 설정 및 API 키 로드 ---
load_dotenv()
MOLIT_API_KEY = os.getenv("MOLIT_API_KEY") # .env 파일에 MOLIT_API_KEY=인증키 형식으로 저장

# --- 2. 국토부 API 요청 정보 ---
BASE_URL = " http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=2086&style_num=713&start_dt=2024&end_dt=2024"

# 요청 파라미터 (예제 기반)
# form_id: 통계표 ID (2086 = 미분양현황_종합)
# style_num: 서식 번호 (713)
# start_dt, end_dt: 조회 기간 (YYYY) -> 월별 데이터는 다른 파라미터 필요 가능성 있음
PARAMS = {
    "key": MOLIT_API_KEY,
    "form_id": "2086",      # 미분양현황_종합
    "style_num": "713",
    "start_dt": "2015",     # 시작 년도 (최소 5년치 요청)
    "end_dt": "2024",       # 종료 년도
    # "date_se": "M",       # (추측) 월별 데이터를 원할 경우 필요할 수 있는 파라미터
    # "start_month": "01",  # (추측)
    # "end_month": "12",    # (추측)
}

# --- 3. 데이터 가져오기 함수 ---
def get_molit_unsold_data(params):
    """국토부 통계누리 API에서 미분양 데이터를 가져와 DataFrame으로 반환"""

    if not params.get("key"):
        print("🚨 오류: MOLIT_API_KEY가 .env 파일에 설정되지 않았습니다.")
        return pd.DataFrame()

    try:
        print("국토부 통계누리 API 요청 중...")
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # HTTP 오류 체크

        # 국토부 API는 JSON 형식이 아닐 수 있음 -> XML 가능성 높음
        # 일단 JSON으로 시도, 실패 시 다른 처리 필요
        try:
            data = response.json() # JSON 파싱 시도
        except json.JSONDecodeError:
            print("🚨 오류: 응답이 JSON 형식이 아닙니다. XML일 가능성이 높습니다.")
            print("--- 응답 내용 (첫 500자) ---")
            print(response.text[:500]) # 실제 응답 내용 확인
            return pd.DataFrame()

        # 정상 응답 확인 (예제 기반)
        status = data.get("result_status", {})
        if status.get("status_code") != "INFO-000":
            print(f"🚨 API 오류: {status.get('message')} (코드: {status.get('status_code')})")
            return pd.DataFrame()

        # 실제 데이터 추출 (예제 기반)
        form_list = data.get("result_data", {}).get("formList", [])
        if not form_list:
            print("데이터를 찾을 수 없습니다.")
            return pd.DataFrame()

        print("데이터 로드 성공. DataFrame 변환 중...")
        df = pd.DataFrame(form_list)
        
        # 컬럼 이름 및 데이터 타입 정리 (예제 기반)
        df.rename(columns={'date': '연도'}, inplace=True)
        # '미분양(12월기준)' 컬럼 이름이 유효하지 않을 수 있음 -> 실제 응답 확인 필요
        if '미분양(12월기준)' in df.columns:
            df['미분양'] = pd.to_numeric(df['미분양(12월기준)'], errors='coerce')
        else:
            print("경고: '미분양(12월기준)' 컬럼을 찾을 수 없습니다. 실제 컬럼명을 확인하세요.")
            # 다른 컬럼이 실제 미분양 값일 수 있음 (예: 'value', 'data' 등)
            # df['미분양'] = pd.to_numeric(df['실제_미분양_컬럼명'], errors='coerce')
            
        df['연도'] = df['연도'].astype(str) # 연도 데이터를 문자열로 유지 (월별 데이터 아님)

        # 필요한 컬럼만 선택하고 정렬 (예제 기반)
        final_df = df[['연도', '대분류', '구분', '미분양']].sort_values(by=['연도', '대분류', '구분'])

        return final_df

    except requests.exceptions.RequestException as e:
        print(f"🚨 API 요청 중 네트워크 오류 발생: {e}")
    except Exception as e:
        print(f"🚨 처리 중 예상치 못한 오류 발생: {e}")
        
    return pd.DataFrame()

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    
    unsold_data = get_molit_unsold_data(PARAMS)
    
    if not unsold_data.empty:
        print("\n--- 국토부 미분양 현황 데이터 (샘플) ---")
        print(unsold_data.head(10)) # 상위 10개 출력
        print("\n...")
        print(unsold_data.tail(10)) # 하위 10개 출력
    else:
        print("\n데이터 로드에 실패했습니다.")

국토부 통계누리 API 요청 중...
🚨 API 요청 중 네트워크 오류 발생: 500 Server Error: Internal Server Error for url: https://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=2086&style_num=713&start_dt=2024&end_dt=2024&key=0678d88e2630421a9b2d64a23f5ab643+%23+%EA%B5%AD%ED%86%A0%EB%B6%80+%ED%86%B5%EA%B3%84%EB%88%84%EB%A6%AC&form_id=2086&style_num=713&start_dt=2015&end_dt=2024

데이터 로드에 실패했습니다.


In [4]:
import requests  # 인터넷으로 데이터를 요청(request)하기 위해 필요한 도구
import json      # 받아온 데이터를 보기 좋게 정렬하기 위해 필요한 도구

# 1. API 요청에 필요한 정보들
api_key = "0678d88e2630421a9b2d64a23f5ab643"
form_id = "618"    # 통계표 ID (주택건설 인허가실적 총괄)
start_date = "2024"
end_date = "2024"

# 2. API를 호출할 주소(URL) 만들기
url = f" http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=618&style_num=40&start_dt=2014&end_dt=2024"

print(f"'{url}' 주소로 데이터를 요청합니다...")
print("-" * 30)

# 3. requests를 사용해 인터넷으로 데이터 요청
response = requests.get(url)

# 4. 요청 결과 확인 (200이면 "성공")
if response.status_code == 200:
    print("✅ 데이터 요청 성공!")
    
    # 5. 받아온 응답을 JSON 형태로 변환
    data = response.json()
    
    # 6. 필요한 데이터(formList)만 뽑아내기
    data_list = data['result_data']['formList']
    
    # 7. 결과 출력
    print("\n--- [받아온 데이터 목록] ---")
    # json.dumps를 쓰면 딕셔너리를 보기 좋게 출력
    print(json.dumps(data_list, indent=2, ensure_ascii=False))

else:
    # 200이 아니면 "실패"
    print(f"❌ 데이터 요청 실패... (에러 코드: {response.status_code})")

' http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=618&style_num=40&start_dt=2014&end_dt=2024' 주소로 데이터를 요청합니다...
------------------------------
✅ 데이터 요청 성공!

--- [받아온 데이터 목록] ---
[
  {
    "date": "2014",
    "합계": 529100,
    "구분1": "합계",
    "구분2": "합계"
  },
  {
    "date": "2014",
    "합계": 0,
    "구분1": "공공주택",
    "구분2": "국가기관"
  },
  {
    "date": "2014",
    "합계": 129,
    "구분1": "공공주택",
    "구분2": "지자체"
  },
  {
    "date": "2014",
    "합계": 20176,
    "구분1": "공공주택",
    "구분2": "주택사업자"
  },
  {
    "date": "2014",
    "합계": 0,
    "구분1": "공공주택",
    "구분2": "기타"
  },
  {
    "date": "2014",
    "합계": 34123,
    "구분1": "공공주택",
    "구분2": "소계"
  },
  {
    "date": "2014",
    "합계": 13818,
    "구분1": "공공주택",
    "구분2": "LH"
  },
  {
    "date": "2014",
    "합계": 494977,
    "구분1": "민간주택",
    "구분2": "소계"
  },
  {
    "date": "2014",
    "합계": 494977,
    "구분1": "민간주택",
    "구분2": "주택사업자등"
  },
  {
    "date": "2014",
    "합